# Build and test the crop container

This notebook is linked to https://eoap.github.io/mastering-app-package/containers/crop/

## Goal

Create a container and run the crop step in the container image.


## Setup the environment

In [6]:
export WORKSPACE=/workspace/mastering-app-package
export RUNTIME=${WORKSPACE}/runs
rm -fr ${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

## Build the container

Inspect the container file:

In [7]:
cat ${WORKSPACE}/water-bodies/command-line-tools/crop/Dockerfile

FROM docker.io/python:3.10-slim

RUN pip install --no-cache-dir rasterio click pystac loguru pyproj shapely && \
    python -c "import rasterio"

ADD app.py /app/app.py

ENTRYPOINT []


Build the container using `podman`:

In [8]:
podman build --format docker -t localhost/crop:latest ${WORKSPACE}/water-bodies/command-line-tools/crop


STEP 1/4: FROM docker.io/python:3.10-slim
STEP 2/4: RUN pip install --no-cache-dir rasterio click pystac loguru pyproj shapely &&     python -c "import rasterio"
--> Using cache 771703387ab8311730a9448d47d575226b37218987bf7756e9b0a04ebc1a2fdc
--> 771703387ab
STEP 3/4: ADD app.py /app/app.py


--> Using cache aef5f7f9517c155b38e4bcc2cb5ba5ea417f85700559d5862ee51f371ae4f816
--> aef5f7f9517
STEP 4/4: ENTRYPOINT []
--> Using cache 4aa8681dcdb8e3b1222bf92314b6bd005cd41f71ce9f1a8555fec6c0b6695687
COMMIT localhost/crop:latest
--> 4aa8681dcdb
Successfully tagged localhost/crop:latest
4aa8681dcdb8e3b1222bf92314b6bd005cd41f71ce9f1a8555fec6c0b6695687


Show the `crop` help:

In [9]:
podman run --rm -it --env=PYTHONPATH=/app localhost/crop:latest python -m app --help

Usage: app.py [OPTIONS]

  Crops a STAC Item asset defined with its common band name

Options:
  --input-item TEXT  STAC Item URL or staged STAC catalog  [required]
  --aoi TEXT         Area of interest expressed as a bounding box  [required]
  --epsg TEXT        EPSG code  [required]
  --band TEXT        Common band name  [required]
  --help             Show this message and exit.


## Test the crop step in the container

Crop the green band asset:

In [10]:
podman run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/mastering-app-package/runs,target=/runs \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/crop:latest \
    python \
    -m \
    app \
    --aoi \
    "-121.399,39.834,-120.74,40.472" \
    --band \
    green \
    --epsg \
    "EPSG:4326" \
    --input-item \
    https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A

2024-04-11 13:18:02.129 | INFO     | __main__:crop:69 - Read S2B_10TFK_20210713_0_L2A from https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
2024-04-11 13:18:02.806 | INFO     | __main__:crop:72 - Read asset green from https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B03.tif
2024-04-11 13:18:09.578 | INFO     | __main__:crop:90 - Crop https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B03.tif
2024-04-11 13:18:56.293 | INFO     | __main__:crop:112 - Write crop_green.tif
2024-04-11 13:19:01.137 | INFO     | __main__:crop:115 - Done!


Crop the nir band asset:

In [11]:
podman run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/mastering-app-package/runs,target=/runs \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/crop:latest \
    python \
    -m \
    app \
    --aoi \
    "-121.399,39.834,-120.74,40.472" \
    --band \
    nir \
    --epsg \
    "EPSG:4326" \
    --input-item \
    https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A

2024-04-11 13:19:06.260 | INFO     | __main__:crop:69 - Read S2B_10TFK_20210713_0_L2A from https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
2024-04-11 13:19:06.932 | INFO     | __main__:crop:72 - Read asset nir from https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B08.tif
2024-04-11 13:19:09.389 | INFO     | __main__:crop:90 - Crop https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B08.tif
2024-04-11 13:20:21.714 | INFO     | __main__:crop:112 - Write crop_nir.tif
2024-04-11 13:20:26.459 | INFO     | __main__:crop:115 - Done!


List the outputs:

In [12]:
tree ${RUNTIME}

/workspace/mastering-app-package/runs
├── crop_green.tif
└── crop_nir.tif

0 directories, 2 files
